In [29]:
import os
import cv2

# Define the emotions in the dataset
emo_dict = ["happiness", "anger", "sadness", "disgust", "fear", "interest"]

# Define the path to the dataset directory
dataset_path = "F:\\FYP\\Dataset\\baum1s\\acted"
frame_directory = "F:\\FYP\\Dataset\\baum1s\\frames"
# Initialize lists to store the video data and labels

if not os.path.exists(frame_directory):
  os.makedirs(frame_directory)

# Loop through each subdirectory that represents an emotion class
for emotion_dir in os.listdir(dataset_path):
    # Get the label from the directory name (e.g. "happiness")
    if emotion_dir in emo_dict:
        label = emotion_dir
    else:
        continue
    # Loop through each video file in the subdirectory
    for file in os.listdir(os.path.join(dataset_path, emotion_dir)):
        # Read video file as numpy array
        video = cv2.VideoCapture(os.path.join(dataset_path, emotion_dir, file))
        frames = []
        frame_count = 0
        while video.isOpened():
            ret, frame = video.read()
            if ret:
                # Convert image to RGB format
                frame = cv2.resize(frame, (224, 224))
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frames.append(frame)
                
                # Save frame as an image file
                frame_filename = os.path.join(frame_directory, label, f"{file}_{frame_count}.jpg")
                os.makedirs(os.path.dirname(frame_filename), exist_ok=True)
                cv2.imwrite(frame_filename, frame)
                
                frame_count += 1
            else:
                break
        video.release()


In [1]:
import os
import cv2
import numpy as np

# Define the emotions in the dataset
emo_dict = ["happiness", "anger", "sadness", "disgust", "fear", "interest"]

# Define the path to the frames directory
frame_path = "F:\\FYP\\Dataset\\baum1s\\frames"

# Initialize lists to store the frame data and labels
frame_data = []
frame_labels = []

# Loop through each subdirectory that represents an emotion class
for emotion_dir in os.listdir(frame_path):
    if emotion_dir in emo_dict:
        label = emotion_dir
    else:
        continue
    # Loop through each image file in the subdirectory
    for file in os.listdir(os.path.join(frame_path, emotion_dir)):
        # Read image file as numpy array
        image = cv2.imread(os.path.join(frame_path, emotion_dir, file))
        # Convert image to RGB format
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Append image data and label to lists
        frame_data.append(image)
        frame_labels.append(label)

In [2]:

# Convert lists to numpy arrays
frame_data = np.array(frame_data)
frame_labels = np.array(frame_labels)

In [3]:
import pandas as pd

df = pd.DataFrame(frame_labels, columns=['Label'])
df['Label'].replace({'happiness': 0, 'anger': 1, 'sadness': 2, 'disgust': 3, 'fear': 4, 'interest': 5}, inplace=True)

frame_labels = df['Label'].to_list()


In [4]:
print(frame_data.shape)
print(len(frame_labels))
print(frame_labels[:10])

(23823, 224, 224, 3)
23823
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [5]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout, LSTM, Flatten
from keras.layers import TimeDistributed
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16

In [6]:
# Convert labels to one-hot encoded format
labels = to_categorical(frame_labels, num_classes=len(emo_dict))

# Load pre-trained VGG16 model
vgg16_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze the weights of the pre-trained layers
for layer in vgg16_model.layers:
    layer.trainable = False

# Add custom classification layers on top of the pre-trained layers
x = vgg16_model.output
x = TimeDistributed(Flatten())(x)
x = LSTM(256, return_sequences=False)(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(len(emo_dict), activation="softmax")(x)

# Create a new model with the pre-trained layers and custom layers
model = Model(inputs=vgg16_model.input, outputs=x)

# Compile the model with categorical crossentropy loss and Adam optimizer
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model on the dataset for 15 epochs with batch size of 32
model.fit(frame_data, labels, epochs=15, batch_size=32)


Epoch 1/15
745/745 [==============================] - 4470s 6s/step - loss: 0.9769 - accuracy: 0.6384
Epoch 2/15
745/745 [==============================] - 4559s 6s/step - loss: 0.5100 - accuracy: 0.8198
Epoch 3/15
745/745 [==============================] - 4832s 6s/step - loss: 0.3809 - accuracy: 0.8675
Epoch 4/15
745/745 [==============================] - 4944s 7s/step - loss: 0.3186 - accuracy: 0.8886
Epoch 5/15
745/745 [==============================] - 4757s 6s/step - loss: 0.3078 - accuracy: 0.8938
Epoch 6/15
745/745 [==============================] - 4609s 6s/step - loss: 0.2231 - accuracy: 0.9233
Epoch 7/15
745/745 [==============================] - 4620s 6s/step - loss: 0.2132 - accuracy: 0.9240
Epoch 8/15
745/745 [==============================] - 4571s 6s/step - loss: 0.1886 - accuracy: 0.9365
Epoch 9/15
745/745 [==============================] - 4561s 6s/step - loss: 0.1736 - accuracy: 0.9411
Epoch 10/15
745/745 [==============================] - 4568s 6s/step - loss: 0.170

In [7]:
model.save("F:\\FYP\\Dataset\\train\\vgg16.h5")

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [8]:
import tensorflow as tf

In [11]:
# Load the .h5 file as a Keras model
model = tf.keras.models.load_model('F:\\FYP\\Dataset\\train\\vgg16.h5')
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [18]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model('F:\\FYP\\Dataset\\train\\') # path to the SavedModel directory
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [19]:
# Convert the model to a .tflite file
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False
tflite_model = converter.convert()
tflite_model = converter.convert()

# Save the .tflite file
with open('F:\\FYP\\Dataset\\train\\model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Moin\AppData\Local\Temp\tmp8fbsd_5q\assets


INFO:tensorflow:Assets written to: C:\Users\Moin\AppData\Local\Temp\tmp8fbsd_5q\assets


INFO:tensorflow:Assets written to: C:\Users\Moin\AppData\Local\Temp\tmpt_sct168\assets


INFO:tensorflow:Assets written to: C:\Users\Moin\AppData\Local\Temp\tmpt_sct168\assets
